In [ ]:
import json

def parse_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # 打印整个JSON结构
    print(json.dumps(data, indent=4))

    # 进行其他操作，例如提取特定字段、遍历数据等

# 替换为你的JSON文件路径
json_file_path = '/home/chenzhixuan/Workspace/R2Gen/data/iu_xray/annotation.json'

parse_json_file(json_file_path)

# 统计数据集元素个数

In [ ]:
import json

def count_elements_in_json(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
        train_count = len(data['train'])
        val_count = len(data['val'])
        test_count = len(data['test'])
    return train_count, val_count, test_count

json_file_path = '/jhcnas1/chenzhixuan/mimic-cxr-jpg/annotation.json'
train_count, val_count, test_count = count_elements_in_json(json_file_path)
print('train_count: ', train_count)
print('val_count: ', val_count)
print('test_count: ', test_count)

# 测试DataParallel

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import DataParallel
from torch.utils.data import DataLoader
import torchvision.models as models
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# 1. 定义模型
class Child_MyModel(nn.Module):
    def __init__(self):
        super(Child_MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x, y):
        return self.fc(x+y)

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)
        self.model = models.resnet50(pretrained=True)

    def forward(self, x):
        return self.model(x)

# 2. 封装模型
device_ids = [0, 1]  # 使用编号为 0 和 1 的 GPU 设备
model = MyModel()
model.cuda()
model = DataParallel(model, device_ids=device_ids)

# 4. 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 5. 训练模型
for epoch in range(10):  # 迭代 10 次
    for i in range(100):  # 每次迭代有 100 个批次
        # 生成随机输入数据和标签
        inputs = torch.randn(2, 3, 224, 224).to('cuda:0')  # 假设每个批次有 32 个样本，每个样本是 3x224x224 的图像
        labels = torch.randn(2, 1000).to('cuda:0')  # 假设每个样本有 1000 个类别

        # 清零梯度
        optimizer.zero_grad()

        # 正向传播
        outputs = model(inputs)

        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()

        # 打印损失
        if i % 10 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')



In [ ]:
from models.r2gen import R2GenModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import DataParallel
from torch.utils.data import DataLoader
import torchvision.models as models
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"


# 2. 封装模型
device_ids = [0, 1]  # 使用编号为 0 和 1 的 GPU 设备
model = R2GenModel(0,0)
model.cuda()
model = DataParallel(model, device_ids=device_ids)

# 4. 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 5. 训练模型
for epoch in range(10):  # 迭代 10 次
    for i in range(100):  # 每次迭代有 100 个批次
        # 生成随机输入数据和标签
        inputs = torch.randn(5, 2, 3, 224, 224).to('cuda:0')  # 假设每个批次有 32 个样本，每个样本是 3x224x224 的图像
        labels = torch.randn(2, 1000).to('cuda:0')  # 假设每个样本有 1000 个类别

        # 清零梯度
        optimizer.zero_grad()

        # 正向传播
        outputs = model(inputs, labels, mode='train')

        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()

        # 打印损失
        if i % 10 == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')



In [16]:
# cumulative BLEU scores
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'small', 'test']]
candidate = ['this', 'is', 'a', 'test']
print('Cumulative 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.750000
Cumulative 2-gram: 0.500000
Cumulative 3-gram: 0.000000
Cumulative 4-gram: 0.000000


/home/chenzhixuan/anaconda3/envs/r2gen/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/chenzhixuan/anaconda3/envs/r2gen/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [26]:
# 导入模型权重
import torch
import torch.nn as nn
checkpoint = torch.load('/home/chenzhixuan/Workspace/R2Gen/results/CTRG_slice10/model_best.pth')
checkpoint['epoch']

32

In [1]:
import torch

x = torch.tensor([[1, 2],
                  [3, 4]])

# 在行维度上重复两次，列维度上重复三次
y = x.repeat(2, 1)

print(y)

tensor([[1, 2],
        [3, 4],
        [1, 2],
        [3, 4]])
